# Clase 5: Estadística Descriptiva (SQL)

Más allá de listar datos, necesitamos resumirlos. SQL es poderosísimo para calcular estadísticas básicas instantáneamente.

## Objetivo de Aprendizaje
- Funciones de Agregación: `COUNT`, `SUM`, `AVG`.
- Entender la **Desviación Estándar** (Volatilidad) para medir consistencia.

---

## Configuración (Carga de Datos)

In [ ]:
from google.colab import files
import os
import pandas as pd
!pip install pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

if not os.path.exists("/root/.kaggle/kaggle.json"):
    if not os.path.exists("kaggle.json"):
        files.upload()
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

if not os.path.exists("baseball_data"):
    !kaggle datasets download -d open-source-sports/baseball-databank
    !unzip -q baseball-databank.zip -d baseball_data

batting = pd.read_csv('baseball_data/Batting.csv')

## Paso 1: Agregaciones Básicas (Promedios)

¿Cuál es el promedio de Home Runs por temporada en toda la historia de la liga?

In [ ]:
query_avg = """
SELECT 
    AVG(HR) as Promedio_HR,
    MAX(HR) as Maximo_HR,
    SUM(HR) as Total_HR_Historia
FROM batting;
"""
pysqldf(query_avg)

## Paso 2: Agrupando Datos (`GROUP BY`)

Los promedios globales esconden detalles. Calculemos el promedio de HR **por Equipo** en el año 2015.

In [ ]:
query_group = """
SELECT 
    teamID, 
    AVG(HR) as Promedio_HR_Jugador,
    SUM(HR) as Total_HR_Equipo
FROM batting
WHERE yearID = 2015
GROUP BY teamID
ORDER BY Total_HR_Equipo DESC
LIMIT 5;
"""
pysqldf(query_group)

## Paso 3: Midiendo la Consistencia (Desviación Estándar)

Aquí entra la magia. Un jugador que batea siempre 20 HR es consistente. Uno que batea 0 un año y 40 el otro es volátil.

*Nota: `pandasql` usa SQLite, el cual a veces requiere extensiones para `STDDEV`. Si falla, usaremos Pandas nativo.*

In [ ]:
# Analicemos la carrera de un jugador famoso, por ejemplo 'Barry Bonds' (bondsba01)
player_id = 'bondsba01'

batting_bonds = batting[batting['playerID'] == player_id]

# Calculamos metricas con Pandas
media = batting_bonds['HR'].mean()
desviacion = batting_bonds['HR'].std()

print(f"Jugador: {player_id}")
print(f"Promedio HR por temporada: {media:.2f}")
print(f"Desviación Estándar (Volatilidad): {desviacion:.2f}")

if desviacion > 10:
    print("Conclusión: Rendimiento altamente variable (picos altos y bajos).")
else:
    print("Conclusión: Jugador consistente.")